## Phys 704 Final Project

 ##### Name: Michael Bouliane 
 ##### ID: 20801403

## Setting Up Ising Model Class

Ising hamiltonian defined as:

$$ H = -J\sum_{\langle i,j \rangle} \sigma_{i} \sigma_{j} $$

Where $J=1$ and $\sigma_{i} = \pm 1$. Lattice is a square lattice of size $L \times L$, where spins are initialized in random configurations.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import multiprocess as mp

class IsingSystem():
    def __init__(self,T=1.,L=5):
        '''Initializes an Ising lattice of size LxL with reciprocal temperature T'''
        
        self.temp = T
        self.lattice_size = L

        self.lattice = np.random.choice((-1,1), size=(self.lattice_size, self.lattice_size))
    
    def energy(self,idx):
        '''Computes the energy of a single spin site at given location
        
        Args:
        idx: Site index of spin

        Returns:
        E: Energy of site computed with Ising hamiltonian'''

        site = self.lattice[idx]

        left = (idx[0], idx[1]-1)
        right = (idx[0], (idx[1]+1)%self.lattice_size)
        top = (idx[0]-1, idx[1])
        bottom = ((idx[0]+1)%self.lattice_size, idx[1])

        E = -site*sum((self.lattice[left], self.lattice[right], self.lattice[top], self.lattice[bottom]))
        
        return(E)

    def magnetization(self):
        
        M = np.sum(self.lattice)
        M /= self.lattice_size**2

        return(abs(M))
    
    def energy_per_site(self):
        E = 0.

        for i in range(self.lattice_size):
            for j in range(self.lattice_size):
                E += self.energy((i,j))
        
        E /= self.lattice_size**2

        return(E)

    def metropolis(self, steps, accept_prob):

        steps = int(steps)*self.lattice_size**2 
        accept_steps = int(accept_prob*steps)
                
        idxs = np.random.randint(0,self.lattice_size,(steps,2)) #random sites for spin flips
        probs = np.random.random(size=steps) #array of acceptance probabilities

        energy_array = []
        energy = 0.
        energy2 = 0.
        ei = 0.

        mag_array = []
        mag = 0.
        mag2 = 0.
        mi = 0.

        n = 0

        for i in range(steps):
            idx = tuple(idxs[i,:])
            energy_init = self.energy(idx)
            energy_final = -energy_init

            dE = energy_final-energy_init

            # if final state is lower in energy, accept the spin flip
            if dE <= 0: 
                self.lattice[idx] = -self.lattice[idx]

            else:
                boltz_fact = np.exp(-dE/self.temp) #boltzmann weight of energy difference

                # accept the spin flip that increases energy if boltzmann weight > acceptance prob
                if probs[i]<boltz_fact:
                    self.lattice[idx] = -self.lattice[idx]

                if i % 1000 == 0:
                    ei = self.energy_per_site()
                    mi = self.magnetization()

                    energy_array.append(ei)
                    mag_array.append(mi)


                    if i >= accept_steps:
                        energy += ei
                        energy2 += ei*ei

                        mag += mi
                        mag2 += mi*mi

                        n += 1

        return(np.array(energy_array),
            energy/n,
            energy2/n,
            np.array(mag_array),
            mag/n,
            mag2/n)

    
    def heat_cycle(self, steps, prob):
        
        temps = np.flip(np.concatenate((np.linspace(1, 2, 10), np.linspace(2, 3, 30), np.linspace(3, 4, 10))))
        
        energy = np.zeros(50)
        energy2 = np.zeros(50)

        mag = np.zeros(50)
        mag2 = np.zeros(50)

        for T in range(50):
            self.temp = temps[T]
            _, energy[T], energy2[T], _, mag[T], mag2[T] = self.metropolis(steps, prob)
        
        return(energy, energy2, mag, mag2, temps)    

In [ ]:
test = IsingSystem(1, 16)

energy, e, e2, mag, m, m2 = test.metropolis(1e5, 0.1)

plt.plot(energy)
plt.show()

plt.plot(mag)
plt.show()

In [ ]:
global finite_scale
def finite_scale(L):
    return([IsingSystem(1, L).heat_cycle(1e4, 0.1)])

Ls = [12, 16, 20, 24]
pool = mp.Pool(4) 
results = pool.map(finite_scale, Ls)
pool.close()

print(results)